In [1]:
import h5py
import numpy as np
import os
import torch

# from tensorflow.keras.layers import Input,Reshape,ZeroPadding2D,MaxPool2D,Conv1D,Conv2D,Dropout,Flatten,Dense,Activation,MaxPooling2D,AlphaDropout
# from tensorflow.keras import layers
# import tensorflow.keras.models as Model
# from tensorflow.keras.regularizers import *
# from tensorflow.keras.optimizers import Adam
# import seaborn as sns
# import tensorflow as tf
# import matplotlib.pyplot as plt
# from tensorflow.keras.models import load_model
# import gc

In [2]:
raw_data = h5py.File('../ExtractDataset/part0.h5')
sample_size = raw_data['X'].shape[0]
sample_split = int(sample_size*0.8)
idx = np.random.permutation(sample_size)
train_idx, test_idx = idx[:sample_split], idx[sample_split:]
X_train = np.array(raw_data['X'][:][train_idx])
Y_train = np.argmax(np.array(raw_data['Y'][:][train_idx]), axis=1)
Z_train = np.array(raw_data['Z'][:][train_idx])
print(X_train.shape)
print(Y_train.shape)
print(Z_train.shape)
X_test = np.array(raw_data['X'][:][test_idx])
Y_test = np.argmax(np.array(raw_data['Y'][:][test_idx]), axis=1)
Z_test = np.array(raw_data['Z'][:][test_idx])
print(X_test.shape)
print(Y_test.shape)
print(Z_test.shape)
raw_data.close()

for i in range(1,12):
    filename = '../ExtractDataset/part'+str(i) + '.h5'
    print(filename)
    raw_data = h5py.File(filename,'r')
    sample_size = raw_data['X'].shape[0]
    sample_split = int(sample_size*0.8)
    idx = np.random.permutation(sample_size)
    train_idx, test_idx = idx[:sample_split], idx[sample_split:]
    X_train = np.vstack((X_train, np.array(raw_data['X'][:][train_idx])))
    Y_train = np.hstack((Y_train, np.argmax(np.array(raw_data['Y'][:][train_idx]), axis=1)))
    Z_train = np.vstack((Z_train, np.array(raw_data['Z'][:][train_idx])))
    X_test = np.vstack((X_test, np.array(raw_data['X'][:][test_idx])))
    Y_test = np.hstack((Y_test, np.argmax(np.array(raw_data['Y'][:][test_idx]), axis=1)))
    Z_test = np.vstack((Z_test, np.array(raw_data['Z'][:][test_idx])))
    raw_data.close()

(24960, 1024, 2)
(24960,)
(24960, 1)
(6240, 1024, 2)
(6240,)
(6240, 1)
../ExtractDataset/part1.h5
../ExtractDataset/part2.h5
../ExtractDataset/part3.h5
../ExtractDataset/part4.h5
../ExtractDataset/part5.h5
../ExtractDataset/part6.h5
../ExtractDataset/part7.h5
../ExtractDataset/part8.h5
../ExtractDataset/part9.h5
../ExtractDataset/part10.h5
../ExtractDataset/part11.h5


In [3]:
X_train = X_train.reshape((X_train.shape[0], 1, 1024, 2))
X_test = X_test.reshape((X_test.shape[0], 1, 1024, 2))
print('X_train-size：',X_train.shape)
print('Y_train-size：',Y_train.shape)
print('Z_train-size：',Z_train.shape)
print('X_test-size：',X_test.shape)
print('Y_test-size：',Y_test.shape)
print('Z_test-size：',Z_test.shape)

X_train-size： (299520, 1, 1024, 2)
Y_train-size： (299520,)
Z_train-size： (299520, 1)
X_test-size： (74880, 1, 1024, 2)
Y_test-size： (74880,)
Z_test-size： (74880, 1)


In [4]:
from torch.utils.data import TensorDataset, DataLoader
training_set = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
train_dataloader = DataLoader(training_set, batch_size=16, shuffle=True)
testing_set = TensorDataset(torch.Tensor(X_test), torch.Tensor(Y_test))
test_dataloader = DataLoader(testing_set, batch_size=16, shuffle=False)

In [5]:
from torch import nn
from torch.nn import functional as F

classes = ['32PSK',
 '16APSK',
 '32QAM',
 'FM',
 'GMSK',
 '32APSK',
 'OQPSK',
 '8ASK',
 'BPSK',
 '8PSK',
 'AM-SSB-SC',
 '4ASK',
 '16PSK',
 '64APSK',
 '128QAM',
 '128APSK',
 'AM-DSB-SC',
 'AM-SSB-WC',
 '64QAM',
 'QPSK',
 '256QAM',
 'AM-DSB-WC',
 'OOK',
 '16QAM']


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv0 = nn.Conv2d(1, 16, kernel_size=(3, 2), padding='valid')
        self.bn0 = nn.BatchNorm2d(16)
        self.conv1 = nn.Conv2d(16, 32, kernel_size=(3, 1), padding='same')
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 1), padding='same')
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=(3, 1), padding='same')
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=(3, 1), padding='same')
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 64, kernel_size=(3, 1), padding='same')
        self.bn5 = nn.BatchNorm2d(64)
        self.conv6 = nn.Conv2d(64, 64, kernel_size=(3, 1), padding='same')
        self.bn6 = nn.BatchNorm2d(64)
        self.conv7 = nn.Conv2d(64, 32, kernel_size=(3, 1), padding='same')
        self.bn7 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(4064, 24)
        self.relu = nn.ReLU()
        self.pooling = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))

    def forward(self, x):
        x = self.conv0(x)
        x = self.bn0(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu(x)
        
        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu(x)
        
        x = self.conv7(x)
        x = self.bn7(x)
        x = self.relu(x)
        
        x = F.avg_pool2d(x, x.size()[3])
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        return x

net = Net()

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
global_loss = []
global_accu = []
num_epochs = 50
import time

ts = time.time()
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        labels = labels.type(torch.long)
#         print(inputs, labels, outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
#         print(inputs, labels, predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    global_loss.append(running_loss)
    global_accu.append(100 * correct / total)
    print('[Epoch %d] loss: %f' %
          (epoch + 1, running_loss))
    print('Accuracy of the network: %f %%' % (
        100 * correct / total))

tt = time.time()
print('Finished Training, elapsed %.3f seconds'%(tt-ts))

/home/aperture/anaconda3/envs/radioml/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448216815/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch 1] loss: 45369.346259
Accuracy of the network: 28.203459 %


In [ ]:
fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2,figsize=(20,6))

ax1.plot(history.history['accuracy'],'b', history.history['val_accuracy'], 'r')
ax1.set_ylabel('Accuracy Rate',fontsize=12)
ax1.set_xlabel('Iteration',fontsize=12)
ax1.set_title('Categorical Cross Entropy ',fontsize=14)
ax1.legend(['Training Accuracy','Validation Accuracy'],fontsize=12,loc='best')

ax2.plot(history.history['loss'], 'b',history.history['val_loss'],'r')
ax2.set_ylabel('Loss',fontsize=12)
ax2.set_xlabel('Iteration',fontsize=12)
ax2.set_title('Learning Curve ',fontsize=14)
ax2.legend(['Training Loss','Validation Loss'],fontsize=12,loc='best')

# plt.savefig('crosse_results.png')
plt.show()

In [ ]:
model = load_model(filepath)

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
# Plot confusion matrix
batch_size = 1024
test_Y_hat = model.predict(X_test, batch_size=3000)
conf = np.zeros([len(classes),len(classes)])
confnorm = np.zeros([len(classes),len(classes)])
for i in range(0,X_test.shape[0]):
    j = list(Y_test[i,:]).index(1)
    k = int(np.argmax(test_Y_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,len(classes)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
plot_confusion_matrix(confnorm, labels=classes)

In [ ]:
for i in range(len(confnorm)):
    print(classes[i],confnorm[i,i])

In [ ]:

acc={}
Z_test = Z[test_idx]
Z_test = Z_test.reshape((len(Z_test)))
SNRs = np.unique(Z_test)
for snr in SNRs:
    X_test_snr = X_test[Z_test==snr]
    Y_test_snr = Y_test[Z_test==snr]
    
    pre_Y_test = model.predict(X_test_snr)
    conf = np.zeros([len(classes),len(classes)])
    confnorm = np.zeros([len(classes),len(classes)])
    for i in range(0,X_test_snr.shape[0]):    #该信噪比下测试数据量
        j = list(Y_test_snr[i,:]).index(1)   #正确类别下标
        k = int(np.argmax(pre_Y_test[i,:])) #预测类别下标
        conf[j,k] = conf[j,k] + 1
    for i in range(0,len(classes)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
   
    plt.figure()
    plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
    
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print ("Overall Accuracy %s: "%snr, cor / (cor+ncor))
    acc[snr] = 1.0*cor/(cor+ncor)

In [ ]:

plt.plot(acc.keys(),acc.values())
plt.ylabel('ACC')
plt.xlabel('SNR')
plt.grid(True)
plt.show()